In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
import torchvision
from torchvision import datasets, transforms, models
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from collections import OrderedDict
from PIL import Image
import seaborn as sns
import splitfolders
from IPython.display import display, HTML

import torchtext
from torchtext.data import get_tokenizer   # for tokenization
from collections import Counter     # for tokenizer


In [2]:
# splitfolders.ratio("Food Images", output="food_images", 
#                    seed=30, ratio=(.7, .2, .1), 
#                    group_prefix=None)

In [3]:
data_df = pd.read_csv('Food Ingredients and Recipe Dataset with Image Name Mapping.csv')
data_df.iloc[1,4]

'crispy-salt-and-pepper-potatoes-dan-kluger'

In [4]:
data_dir = './food_images'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/val'
test_dir = data_dir + '/test'

In [5]:
# img,recipe_tokens = torch_data[10]

# fig, ax = plt.subplots()
# img


In [6]:
# [torch_data.vocab.itos[token] for token in recipe_tokens.tolist()]

In [7]:
# image_data_ref = pd.read_csv('Food Ingredients and Recipe Dataset with Image Name Mapping.csv')
# image_data_ref.ndim
# display(image_data_ref)


In [8]:
##recipe

In [10]:
from dataloader import *
from model import *

train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(256),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485,0.456,0.406],
                                                           [0.229,0.224,0.225])])
torch_data = CustomDataset(recipe_csv='Food Ingredients and Recipe Dataset with Image Name Mapping.csv',
                                    img_dir='Food Images', transform=train_transforms)

path = 'my_checkpoint.pth.tar'
checkpoint = torch.load(path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Hyperparameters
embed_size = 256
hidden_size = 256
vocab_size = len(torch_data.vocab)
num_layers = 2
learning_rate = 3e-4
num_epochs = 1

model = CNNtoRNN(embed_size, hidden_size, vocab_size, num_layers).to(device)
model.load_state_dict(checkpoint['state_dict'])

# No use
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']  # ALREADY DEFFINED ABOE
# loss = checkpoint['loss']

C:\Users\erice\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\erice\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [14]:
img,recipe_tokens = torch_data[1]
model.recipe_generate(img.unsqueeze(0), torch_data.vocab, max_length = 100)



['<START>',
 'blend',
 'first',
 'flour',
 ',',
 'and',
 'remaining',
 '4',
 'zest',
 'in',
 'a',
 'bowl',
 'with',
 'just',
 'oil',
 'over',
 'high',
 'heat',
 'until',
 'almost',
 'thick',
 ',',
 'turning',
 'occasionally',
 ',',
 'about',
 '1',
 'minute',
 '.',
 'add',
 'remaining',
 'sugar',
 ',',
 'garlic',
 ',',
 'peas',
 ',',
 'cream',
 ',',
 'and',
 '2',
 'tablespoons',
 'salt',
 '.',
 'stir',
 'in',
 '1/4',
 'teaspoon',
 'pepper',
 '.',
 'add',
 'sage',
 'and',
 'cook',
 'until',
 'softened',
 ',',
 'stirring',
 'occasionally',
 ',',
 'on',
 'to',
 '375°f',
 ',',
 'about',
 '6',
 'minutes',
 '.',
 'season',
 'in',
 'sauce',
 'and',
 'pepper',
 '.',
 'while',
 'noodles',
 'in',
 'a',
 'platter',
 ',',
 'place',
 '2',
 'tablespoons',
 'oil',
 'in',
 'large',
 'bowl',
 '(',
 'will',
 ')',
 ',',
 'scraping',
 'it',
 'is',
 'dry',
 '.',
 'transfer',
 'to',
 'a',
 'food']

In [ ]:
# torch_data.recipe_df.iloc[1]

Unnamed: 0                                                             1
Title                                    Crispy Salt and Pepper Potatoes
Ingredients            ['2 large egg whites', '1 pound new potatoes (...
Instructions           Preheat oven to 400°F and line a rimmed baking...
Image_Name                    crispy-salt-and-pepper-potatoes-dan-kluger
Cleaned_Ingredients    ['2 large egg whites', '1 pound new potatoes (...
Name: 1, dtype: object